# Binary text classification with a convolutional neural network

In [5]:
# all our imports
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras import layers

# importing the two csv
csv1 = pd.read_csv("data/elonmusk_tweets.csv")
csv2 = pd.read_csv("data/trump_tweets.csv")

In [6]:
# changing the defalt column with to take a better look at the text data
pd.set_option('display.max_colwidth', None)

csv1

,id,created_at,text
0,849636868052275200,2017-04-05T14:56:29,b'And so the robots spared humanity ... https://t.co/v7JUJQWfCv'
1,848988730585096192,2017-04-03T20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exactly. Tesla is absurdly overvalued if based on the past, but that's irr\xe2\x80\xa6 https://t.co/qQcTqkzgMl"""
2,848943072423497728,2017-04-03T16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03T16:30:19,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02T06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat gas fracking. It's basically dead."""
...,...,...,...
2814,142881284019060736,2011-12-03T08:22:07,b'That was a total non sequitur btw'
2815,142880871391838208,2011-12-03T08:20:28,"b'Great Voltaire quote, arguably better than Twain. Hearing news of his own death, Voltaire replied the reports were true, only premature.'"
2816,142188458125963264,2011-12-01T10:29:04,b'I made the volume on the Model S http://t.co/wMCnT53M go to 11. Now I just need to work in a miniature Stonehenge...'
2817,142179928203460608,2011-12-01T09:55:11,"b""Went to Iceland on Sat to ride bumper cars on ice! No, not the country, Vlad's rink in Van Nuys. Awesome family fun :) http://t.co/rBQXJ9IT"""


In [7]:
# some light preprocessing
csv1['id']= 'Trump'
csv1=csv1[['id','text']]
csv1['text'] = csv1['text'].str[2:]

# a function which removes the hyperlinks and the usernames of the featured twitter users
def split_it(string):
    result = re.sub(r'https?://\S+', '', string)
    result1 = re.sub(r'@\S+','',result)
    return result1

csv1['text'] = csv1['text'].apply(split_it)


C:\Users\aleks\AppData\Local\Temp\ipykernel_2136\3275627726.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv1['text'] = csv1['text'].str[2:]
C:\Users\aleks\AppData\Local\Temp\ipykernel_2136\3275627726.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv1['text'] = csv1['text'].apply(split_it)


In [8]:
csv2

,id,text,source
0,763398630812311552,Morning Joe's weakness is its low ratings. I don't watch anymore but I heard he went wild against Rudy Giuliani and #2A - sad &amp; irrelevant!,Twitter for iPhone
1,763391459110313984,.@dbongino You were fantastic in defending both the Second Amendment and me last night on @CNN. Don Lemon is a lightweight - dumb as a rock,Twitter for Android
2,763385288295055360,"""@dbongino: ‘Now cut off my mic!’ Bongino refuses to be bullied by Don Lemon over Trump, 2nd Amendment – HEATED! https://t.co/UgtfaUXzcr""",Twitter for Android
3,763198483927011329,Media desperate to distract from Clinton's anti-2A stance. I said pro-2A citizens must organize and get out vote to save our Constitution!,Twitter for iPhone
4,763180978588221440,"When is the media going to talk about Hillary's policies that have gotten people killed, like Libya, open borders, and maybe her emails?",Twitter Web Client
...,...,...,...
1922,702155147695284224,"""@Vogelsong1: @EdRollins gets it. Was just on @FoxNews explaining how Trump wins the general election."" Thank you Ed!",Twitter for Android
1923,702149896325890048,Ted Cruz lifts the Bible high into the air and then lies like a dog-over and over again! The Evangelicals in S.C. figured him out &amp; said no!,Twitter for Android
1924,702147760926011393,"Ted Cruz does not have the right ""temperment"" to be President. Look at the way he totally panicked in firing his director of comm. BAD!",Twitter for Android
1925,702141081027104769,Wow was Ted Cruz disloyal to his very capable director of communication. He used him as a scape goat-fired like a dog! Ted panicked.,Twitter for Android


In [9]:
# same light preprocessing for the elon data
csv2['id']= 'Elon'
csv2=csv2[['id','text']]

# same function which removes the hyperlinks and the usernames of the featured twitter users
def split_it(string):
    result = re.sub(r'https?://\S+', '', string)
    result1 = re.sub(r'@\S+','',result)
    return result1

csv2['text'] = csv2['text'].apply(split_it)
csv2

C:\Users\aleks\AppData\Local\Temp\ipykernel_2136\2233951345.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv2['text'] = csv2['text'].apply(split_it)


,id,text
0,Elon,Morning Joe's weakness is its low ratings. I don't watch anymore but I heard he went wild against Rudy Giuliani and #2A - sad &amp; irrelevant!
1,Elon,. You were fantastic in defending both the Second Amendment and me last night on Don Lemon is a lightweight - dumb as a rock
2,Elon,""" ‘Now cut off my mic!’ Bongino refuses to be bullied by Don Lemon over Trump, 2nd Amendment – HEATED!"
3,Elon,Media desperate to distract from Clinton's anti-2A stance. I said pro-2A citizens must organize and get out vote to save our Constitution!
4,Elon,"When is the media going to talk about Hillary's policies that have gotten people killed, like Libya, open borders, and maybe her emails?"
...,...,...
1922,Elon,""" gets it. Was just on explaining how Trump wins the general election."" Thank you Ed!"
1923,Elon,Ted Cruz lifts the Bible high into the air and then lies like a dog-over and over again! The Evangelicals in S.C. figured him out &amp; said no!
1924,Elon,"Ted Cruz does not have the right ""temperment"" to be President. Look at the way he totally panicked in firing his director of comm. BAD!"
1925,Elon,Wow was Ted Cruz disloyal to his very capable director of communication. He used him as a scape goat-fired like a dog! Ted panicked.


In [10]:
# here we concatenate both dataframes and change the label variable to binary 
concate_data = pd.concat([csv1,csv2])

concate_data.head()
concate_data["id"] = concate_data["id"].map({'Trump' : 1, 'Elon' : 0})

In [11]:
concate_data.head()

,id,text
0,1,And so the robots spared humanity ...
1,1,"Exactly. Tesla is absurdly overvalued if based on the past, but that's irr\xe2\x80\xa6"
2,1,"Et tu, Walt?'"
3,1,Stormy weather in Shortville ...'
4,1,"Coal is dying due to nat gas fracking. It's basically dead."""


In [12]:
review = concate_data['text'].values
label = concate_data['id'].values

# splitting the data 25% test size
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

In [13]:
# Here we use Tokenizer which allows us to vectorize a text corpus
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  
print(review_train[1])
print(Xcnn_train[1])

Flying car pros: travel in 3D fast. Cons: risk of car falling on head much greater, noisy, grounded in bad weather'
[659, 92, 991, 5, 505, 506, 4, 92, 9, 803, 94, 5, 103]


In [14]:
# Each sequence is the different length of words, we use pad_sequences(), which pads the sequence of words with zeros.
max_lenth = 50

Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=max_lenth)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=max_lenth)
print(Xcnn_train[0, :])

[132 303   1 142 143   2 614  16   9   1 256  98  88  55  35   7  11 168
  52 359 360   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [15]:
# We make the models using layers in it.
# We use 200 dimentions and add my own embeddings, other pre-trained embeddings could be used the most popular one are the GloVe embeddings.
embedding_dim = 200
model= Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=max_lenth))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 200)           1506000   
                                                                 
 conv1d (Conv1D)             (None, 46, 128)           128128    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,635,429
Trainable params: 1,635,429
Non-trainable params: 0
______________________________________________

Unlike image processing text classification is not the primary use of the convolutional neural network but it still works great if we use a 1 dimentional one. Even other stuff like MaxPooling is still applicable!

In [16]:
# Here I fit the model and test the accuracy
model.fit(Xcnn_train, label_train,
                    epochs=10,
                    verbose=False,
                    validation_data=(Xcnn_test, label_test),
                    batch_size=10)
loss, accuracy = model.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9975
Testing Accuracy:  0.9478


Hmm quite a nice result. Either we didn't do enough preprocessing and there're parts of the tweets have great predictive power or our model is just that awsome!